<style>
  .blue-box {
    background-color: #f0f0f0; /* grey background */
    padding: 10px;
    border: 1px solid #ccc;
    text-align: center;
  }
  .blue-text {
    color: #0000ff; /* blue text */
    font-weight: bold;
  }
</style>

<div class="blue-box">
  <h2 class="blue-text">Forecasting VN1 Data at Weekly Time Granularity</h2>
</div>



### Install Libraries

In [ ]:
#Install the necessary libraries for forecasting
!pip install statsforecast mlforecast neuralforecast nixtla -q

Importing Libraries Required for this Notebook

In [ ]:
from nixtla import NixtlaClient
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsforecast import StatsForecast
from statsforecast.models import AutoTBATS, AutoARIMA, AutoETS
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from mlforecast.target_transforms import AutoDifferences
from mlforecast.lag_transforms import (
    RollingMean, RollingStd, RollingMin, RollingMax, RollingQuantile,
    SeasonalRollingMean, SeasonalRollingStd, SeasonalRollingMin,
    SeasonalRollingMax, SeasonalRollingQuantile,ExpandingStd, ExponentiallyWeightedMean,
    ExpandingMean)
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT, NHITS
from neuralforecast.losses.pytorch import MAE
from nixtla import NixtlaClient
from functools import reduce
#import timesfm
import warnings
warnings.filterwarnings('ignore')

### Reading Data

In [ ]:
train = pd.read_excel("VN1_train.xlsx")
test = pd.read_excel("VN1_test.xlsx")
train['ds'] = pd.to_datetime(train['ds'])
test['ds'] = pd.to_datetime(test['ds'])

In [ ]:
train.head()

,unique_id,ds,y,week_of_year_sin,week_of_year_cos,is_thanksgiving_week,is_christmas_week,is_holiday_week,trend,seasonal
0,C0_W100_P10705,2020-07-06,7,-0.176890,-0.984231,0.0,0.0,False,6.924783,0.823216
1,C0_W100_P10705,2020-07-13,7,-0.292057,-0.956401,0.0,0.0,False,6.902250,0.835461
2,C0_W100_P10705,2020-07-20,7,-0.403123,-0.915146,0.0,0.0,False,6.879824,0.866956
3,C0_W100_P10705,2020-07-27,7,-0.508531,-0.861044,0.0,0.0,False,6.857521,0.897566
4,C0_W100_P10705,2020-08-03,7,-0.606800,-0.794854,0.0,0.0,False,6.835363,0.927268


In [ ]:
test.head()

,unique_id,ds,week_of_year_sin,week_of_year_cos,is_thanksgiving_week,is_christmas_week,is_holiday_week,trend,seasonal
0,C0_W100_P10705,2023-10-09,-0.989040,0.147647,0.0,0.0,False,3.275329,-0.748970
1,C0_W100_P10705,2023-10-16,-0.964636,0.263587,0.0,0.0,False,3.275329,-0.376678
2,C0_W100_P10705,2023-10-23,-0.926690,0.375828,0.0,0.0,False,3.275329,-0.699141
3,C0_W100_P10705,2023-10-30,-0.875735,0.482792,0.0,0.0,False,3.275329,-0.326948
4,C0_W100_P10705,2023-11-06,-0.812487,0.582979,0.0,0.0,False,3.275329,-0.649280


# SFFORECAST

### Defining AutoARIMA Parameters

In [ ]:
models = [AutoARIMA(season_length=52)]

sf = StatsForecast(models=models, freq='W-MON', n_jobs=-1)

horizon = 13
sf_fcst = sf.forecast(df=train, h=horizon, X_df=test)

In [ ]:
sf_fcst.head()

In [ ]:
sf_fcst.to_csv('VN1_arima_weekly.csv', index=False)
from google.colab import files
files.download('VN1_arima_weekly.csv')

### Defining AutoTBATS & AutoETS

In [ ]:
models = [AutoTBATS(season_length=52),AutoETS(season_length=52)]

In [ ]:
stat_fcst = StatsForecast(models=models,freq='W-MON', n_jobs=-1)

In [ ]:
train_stat_df_ETS_TBATS = train[["unique_id", "ds", "y"]]

In [ ]:
horizon=13
ets_tbats = stat_fcst.forecast(df=train_stat_df_ETS_TBATS, h=horizon)

In [ ]:
ets_tbats.head()

In [ ]:
ets_tbats.to_csv('VN_1_WEEKLY_ets_tbats_data.csv', index=False)

# MLForecast

#### Using LGBM, XGB, RF

In [ ]:
models = [LinearRegression(),
    lgb.LGBMRegressor(verbosity=-1),
    xgb.XGBRegressor(),
    RandomForestRegressor(random_state=0),
]


mlf = MLForecast(
    models=models,
    freq='W-MON',
    lags=[1,4,8,13,26],

    lag_transforms={
        1: [ExpandingMean()],
        4: [RollingMean(window_size=4), RollingMean(window_size=13)],
        8: [RollingMean(window_size=4), RollingMean(window_size=13)],
        13: [RollingMean(window_size=4), RollingMean(window_size=13)],
    },
    date_features=['year', 'month', 'day', 'quarter'],
    target_transforms=[AutoDifferences(max_diffs=2)])


In [ ]:
mlf.fit(
    train,
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    static_features=[]
)

MLForecast(models=[LinearRegression, LGBMRegressor, XGBRegressor, RandomForestRegressor], freq=W-MON, lag_features=['lag1', 'lag4', 'lag8', 'lag13', 'lag26', 'expanding_mean_lag1', 'rolling_mean_4_lag4', 'rolling_mean_13_lag4', 'rolling_mean_4_lag8', 'rolling_mean_13_lag8', 'rolling_mean_4_lag13', 'rolling_mean_13_lag13'], date_features=['year', 'month', 'day', 'quarter'], num_threads=1)

In [ ]:
future_df=test.copy()

In [ ]:
future_df

,unique_id,ds,week_of_year_sin,week_of_year_cos,is_thanksgiving_week,is_christmas_week,is_holiday_week,trend,seasonal
0,C0_W100_P10705,2023-10-09,-0.989040,0.147647,0.0,0.0,False,3.275329,-0.748970
1,C0_W100_P10705,2023-10-16,-0.964636,0.263587,0.0,0.0,False,3.275329,-0.376678
2,C0_W100_P10705,2023-10-23,-0.926690,0.375828,0.0,0.0,False,3.275329,-0.699141
3,C0_W100_P10705,2023-10-30,-0.875735,0.482792,0.0,0.0,False,3.275329,-0.326948
4,C0_W100_P10705,2023-11-06,-0.812487,0.582979,0.0,0.0,False,3.275329,-0.649280
...,...,...,...,...,...,...,...,...,...
53269,C7_W303_P8795,2023-12-04,-0.456629,0.889657,0.0,0.0,True,-0.631717,1.464974
53270,C7_W303_P8795,2023-12-11,-0.348202,0.937420,0.0,0.0,True,-0.631717,-2.010597
53271,C7_W303_P8795,2023-12-18,-0.234886,0.972023,0.0,0.5,True,-0.631717,-0.980131
53272,C7_W303_P8795,2023-12-25,-0.118273,0.992981,0.0,1.0,False,-0.631717,-1.479564


In [ ]:
horizon=13
mlforecast = mlf.predict(horizon, X_df=future_df)

In [ ]:
mlforecast.head()

,unique_id,ds,LinearRegression,LGBMRegressor,XGBRegressor,RandomForestRegressor
0,C0_W100_P10705,2023-10-09,1.382170,2.833747,2.977500,3.95
1,C0_W100_P10705,2023-10-16,-0.052684,3.244078,3.288966,4.15
2,C0_W100_P10705,2023-10-23,-1.626971,3.194867,3.616523,4.72
3,C0_W100_P10705,2023-10-30,-2.756355,3.722241,3.496357,5.09
4,C0_W100_P10705,2023-11-06,-5.370681,3.609787,4.262606,6.32


In [ ]:
mlforecast.to_csv('VN_1_WEEKLY_mlforecast_data.csv', index=False) 


# Neural Forecast

#### Using TFT and NHITS

In [ ]:
hist_exog_list = ["is_thanksgiving_week",	"is_christmas_week",	"is_holiday_week"]

horizon = len(test)  # Number of future time steps to predict
# Initialize the TFT model
tft_model = TFT(h=13,  # Forecasting horizon of 12 months
                input_size=32,  # Assuming we use 12 months of history
                hist_exog_list=hist_exog_list,
                max_steps=2580,
                start_padding_enabled=True,
                loss=MAE())

# Initialize the NHITS model
nhits_model = NHITS(h=13,  # Forecasting horizon of 12 months
                input_size=32,  # Assuming we use 12 months of history
                hist_exog_list=hist_exog_list,
                max_steps=2580,
                start_padding_enabled=True,
                loss=MAE())

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
nf = NeuralForecast(models=[tft_model, nhits_model], freq='W-MON')  # 'M' for monthly frequency

nf.fit(df=train)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MAE                      | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 1.3 K  | train
4 | temporal_encoder        | TemporalCovariateEncoder | 864 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 256 K  | train
6 | output_adapter          | Linear      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2580` reached.


In [ ]:
neural_fcst = nf.predict(futr_df = future_df)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
neural_fcst.reset_index(inplace = True)

In [ ]:
#Forecast
neural_fcst

,unique_id,ds,TFT,NHITS
0,C0_W100_P10705,2023-10-09,3.004477,2.942177
1,C0_W100_P10705,2023-10-16,2.985394,2.987519
2,C0_W100_P10705,2023-10-23,2.976824,2.926982
3,C0_W100_P10705,2023-10-30,2.973757,2.918899
4,C0_W100_P10705,2023-11-06,2.972291,2.959557
...,...,...,...,...
53269,C7_W303_P8795,2023-12-04,0.417773,0.534775
53270,C7_W303_P8795,2023-12-11,0.394790,0.562368
53271,C7_W303_P8795,2023-12-18,0.375449,0.536171
53272,C7_W303_P8795,2023-12-25,0.365176,0.558145


In [ ]:

from google.colab import files

neural_fcst.to_csv('neural_fcst_VN1_WEEKLY.csv', index=False)
files.download('neural_fcst_VN1_WEEKLY.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# TimeGPT

In [ ]:

# Initialize the NixtlaClient

nixtla_client = NixtlaClient(api_key='Your Key')

# Assuming df is your input dataframe with historical data
# and future_df is your dataframe with future exogenous variables

# Specify the exogenous variables
exog_vars = ["is_thanksgiving_week",	"is_christmas_week",	"is_holiday_we"]

future_df=test.copy()
# Make the forecast
timegpt_fcst_zero = nixtla_client.forecast(df=train, h=13, X_df=future_df, freq='W-MON')


In [ ]:
timegpt_fcst_zero['ds'] = pd.to_datetime(timegpt_fcst_zero['ds'])

In [ ]:
timegpt_fcst_zero.to_csv('timegpt_fcst_zero.csv', index=False)

In [ ]:
timegpt_fcst_zero


,unique_id,ds,TimeGPT
0,C0_W100_P10705,2023-10-09,3.315635
1,C0_W100_P10705,2023-10-16,3.811078
2,C0_W100_P10705,2023-10-23,3.188509
3,C0_W100_P10705,2023-10-30,3.596156
4,C0_W100_P10705,2023-11-06,3.134432
...,...,...,...
53269,C7_W303_P8795,2023-12-04,0.235869
53270,C7_W303_P8795,2023-12-11,0.722868
53271,C7_W303_P8795,2023-12-18,0.866885
53272,C7_W303_P8795,2023-12-25,0.908925


In [ ]:
#Finetuning TimeGPT
timegpt_fcst_finetune_4 = nixtla_client.forecast(df=train, h=13, X_df=future_df, freq='W-MON', finetune_steps=4)

timegpt_fcst_finetune_6 = nixtla_client.forecast(df=train, h=13, X_df=future_df, freq='W-MON', finetune_steps=6)

timegpt_fcst_finetune_8 = nixtla_client.forecast(df=train, h=13, X_df=future_df, freq='W-MON', finetune_steps=8)

In [ ]:
timegpt_fcst_finetune_6.rename(columns={'TimeGPT': 'TimeGPT_6'}, inplace=True)
timegpt_fcst_finetune_4.rename(columns={'TimeGPT': 'TimeGPT_4'}, inplace=True)
timegpt_fcst_finetune_8.rename(columns={'TimeGPT': 'TimeGPT_8'}, inplace=True)

In [ ]:
#Merge All Results
data_frames = [timegpt_fcst_zero,timegpt_fcst_finetune_4, timegpt_fcst_finetune_6, timegpt_fcst_finetune_8 ]
TIMEGPT_forecast_weekly_merged = reduce(lambda  left,right: pd.merge(left,right,on=['unique_id', 'ds'],how='inner'), data_frames)

In [ ]:
TIMEGPT_forecast_weekly_merged.to_csv("VN1_TIMEGPT_forecast_weekly_merged.csv")

## Timesfm

In [ ]:
timesfm_df = pd.read_csv("timesfm_weekly_VN1.csv")

In [ ]:
timesfm_df['ds'] = pd.to_datetime(timesfm_df['ds'])

In [ ]:
#Merge All Results
data_frames = [sf_fcst, ets_tbats,mlforecast, neural_fcst,timegpt_fcst_zero,timesfm_df ]
final_forecast_monthly_merged = reduce(lambda  left,right: pd.merge(left,right,on=['unique_id', 'ds'],how='inner'), data_frames)

In [ ]:
final_forecast_monthly_merged.to_csv("final_forecast_weekly_merged.csv")